<a href="https://colab.research.google.com/github/Cryptix001/ML-BOOTCAMP/blob/main/Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [2]:
# Step 1: Download the dataset
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

# Step 2: Unzip the dataset
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

# Step 3: Define base, train, and validation directories
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Set up paths for cats and dogs in training and validation directories
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

--2024-11-23 00:46:58--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.207, 173.194.203.207, 74.125.199.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   133MB/s    in 0.5s    

2024-11-23 00:46:58 (133 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [3]:
from PIL import Image
import numpy as np
import os

# Define paths to the cats and dogs directories
cats_dir = os.path.join(train_dir, "cats")
dogs_dir = os.path.join(train_dir, "dogs")

# Initialize lists to store training data and labels
x_train = []
y_train = []

# Set the number of examples (you can define this as needed)
NUMBER_OF_EXAMPLES = 1000  # Set this to the desired number of examples

# Image dimensions
img_height, img_width = 224, 224

# Loop through the specified number of images
i = 0
while i < NUMBER_OF_EXAMPLES:
    try:
        if i % 2 == 0:  # Load cat images
            im_path = os.path.join(cats_dir, os.listdir(cats_dir)[i])
            im = Image.open(im_path).convert("RGB")
            im_resized = im.resize((img_width, img_height))
            x_train.append(np.array(im_resized))
            y_train.append(1)  # Label for cat
        else:  # Load dog images
            im_path = os.path.join(dogs_dir, os.listdir(dogs_dir)[i])
            im = Image.open(im_path).convert("RGB")
            im_resized = im.resize((img_width, img_height))
            x_train.append(np.array(im_resized))
            y_train.append(0)  # Label for dog

        i += 1
    except IndexError:
        print(f"Not enough images to reach {NUMBER_OF_EXAMPLES}. Ending early.")
        break

# Convert lists to numpy arrays for model training
x_train = np.array(x_train)
y_train = np.array(y_train)

print("Data processing complete.")
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

Data processing complete.
x_train shape: (1000, 224, 224, 3)
y_train shape: (1000,)


In [4]:
# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers for binary classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Sigmoid for binary classification

# Final model
model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [5]:
# Split data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Convert labels to tensor format for TensorFlow compatibility
y_train = tf.convert_to_tensor(y_train)
y_val = tf.convert_to_tensor(y_val)

In [6]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
import tensorflow as tf

pretrained_model = tf.keras.applications.ResNet50(
    include_top=False,
    input_shape=(150, 150, 3),
    pooling='avg',
    classes=2,
    weights='imagenet'
)

In [8]:
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(x_val, y_val)
)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 219s 8s/step - accuracy: 0.7492 - loss: 0.8835 - val_accuracy: 0.9350 - val_loss: 0.1766
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 200s 8s/step - accuracy: 0.9794 - loss: 0.0554 - val_accuracy: 0.9600 - val_loss: 0.1028
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 199s 8s/step - accuracy: 0.9982 - loss: 0.0119 - val_accuracy: 0.9550 - val_loss: 0.1198
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 200s 8s/step - accuracy: 1.0000 - loss: 0.0047 - val_accuracy: 0.9650 - val_loss: 0.1350
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 201s 8s/step - accuracy: 1.0000 - loss: 8.3698e-04 - val_accuracy: 0.9600 - val_loss: 0.1396
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 197s 8s/step - accuracy: 1.0000 - loss: 5.3964e-04 - val_accuracy: 0.9600 - val_loss: 0.1495
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 199s 8s/step - accuracy: 1.0000 - loss: 2.6639e-04 - val_accuracy: 0.9600 - val_loss: 0.1546
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 202s 8s/step - accuracy: 1.0000 - loss: 2.0558e-04 - val_accu